# Drzewo regresyjne

Wróćmy do przykładu pokazanego podczas lekcji z regresji liniowej.

Zbiór danych: https://www.kaggle.com/datasets/fedesoriano/wind-speed-prediction-dataset

Dokumentacja: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

Parametry tylko dla regresji:

criterion{“squared_error”, “friedman_mse”, “absolute_error”, “poisson”}, default=”squared_error”

In [1]:
from  sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

In [2]:
os.chdir('../')

In [3]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [4]:
# Wczytanie danych
df = pd.read_csv("data/wind_dataset.csv")

In [ ]:
# nagłówek
df.head()

In [ ]:
# Typy danych
df.info()

In [7]:
# Zmiana typu daty
df['DATE'] = pd.to_datetime(df['DATE'])

In [8]:
# Dodamy kilka informacji o okresie w trakcie roku
df['month'] = df['DATE'].dt.month
df['quarter'] = df['DATE'].dt.quarter
df['day'] = df['DATE'].dt.day

In [9]:
# Podział na zbior treningowy i testowy wykonamy po datach
# Często przyjmuje się założenie - uczymy na starszych danych, testujemy na nowszych
threshold = df['DATE'].quantile(0.7)
train = df[df['DATE']<=threshold].reset_index(drop=True)
test = df[df['DATE']> threshold].reset_index(drop=True)

In [ ]:
threshold

In [ ]:
# kolumny
df.columns

In [12]:
# zmienne do modelu
x_names = df.columns[2:]

In [13]:
# Podział na zbiory
train_x = train[x_names]
test_x = test[x_names]
train_y = train['WIND']
test_y = test['WIND']

In [14]:
# Estymacja modelu
model = DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=30, ccp_alpha=0.1).fit(train_x,train_y)

In [ ]:
# ważność zmiennych
model.feature_importances_

In [ ]:
# nazwy zmiennych
model.feature_names_in_

In [17]:
# Usunięcie zbędnych kolumn
for i in ['IND','IND.1','IND.2','day']:
    del train_x[i]
    del test_x[i]


In [18]:
# Estymacja modelu
model = DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=30, ccp_alpha=0.1).fit(train_x,train_y)

In [19]:
# predykcje
train_pred = model.predict(train_x)
test_pred= model.predict(test_x)
total_pred = model.predict(df[model.feature_names_in_])

In [ ]:
# Wykres rozrzutu
plt.scatter(total_pred, df['WIND'])
plt.plot(np.arange(0,df['WIND'].max()+1),np.arange(0,df['WIND'].max()+1),color='red')
plt.show()

In [ ]:
df['pred'] = total_pred
df.groupby('pred')['WIND'].mean()

In [ ]:
test['pred'] = test_pred
test.groupby('pred')['WIND'].mean()

Wykres rozrzutu wygląda na osi X jak skończona liczba punktów, co jest logiczne, z uwagi na skonczoną liczbę liści.

Teraz narysujemy drzewo.

dokumentacja: https://scikit-learn.org/1.5/modules/generated/sklearn.tree.plot_tree.html

In [23]:
from sklearn.tree import plot_tree

In [ ]:
# wykres drzewa
plt.figure(figsize=[15,12])
plot_tree(model, feature_names=model.feature_names_in_, filled=True)
plt.show()

In [ ]:
# MAE train
mean_absolute_error(train_y, train_pred)

In [ ]:
# MAE test
mean_absolute_error(test_y, test_pred)

In [27]:
# Stwórzmy model bez parametru ccp_alpha

model_2 = DecisionTreeRegressor(max_depth=10, min_samples_leaf=20, min_samples_split=30).fit(train_x, train_y)

In [28]:
# predykcje

train_pred_2 = model_2.predict(train_x)
test_pred_2 = model_2.predict(test_x) 

In [ ]:
# MAE train
mean_absolute_error(train_y, train_pred_2)

In [ ]:
# MAE test
mean_absolute_error(test_y, test_pred_2)

In [ ]:
# R2
model_2.score(test_x,test_y)

In [ ]:
# z modelu możemy odczytać parametry
model.criterion

In [ ]:
model_2.get_n_leaves()

Chociaż na zbiorze treningowym błąd się zmniejszył, to na zbiorze testowym jest wyższy, lepiej wybrać mniej złożony model. 
Aby porównać model drzewa do regresji liniowe, Należałoby wytrenować i przetestować modele na tym samym zbiorze.

In [34]:
# porównanie z regresją liniową 
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression().fit(train.dropna()[model.feature_names_in_],train.dropna()['WIND'])

In [35]:
# predykcje
l_train_pred = linear_model.predict(train_x.dropna())
l_test_pred = linear_model.predict(test_x.dropna())

In [ ]:
# MAE test
mean_absolute_error(test.dropna()['WIND'], l_test_pred)